In [22]:
%matplotlib inline
import bisect
import copy 
import os 
from collections import deque, Counter
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import editdistance
import sys
import RNA
from typing import Dict, List, Tuple

# import path 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.sequence_utils import translate_one_hot_to_string,generate_random_mutant
from utils.sequence_utils import translate_string_to_one_hot, translate_one_hot_to_string
from models.Theoretical_models import *
from models.Noise_wrapper import *
from exploration_strategies.CE import *
from utils.landscape_utils import *
from models.RNA_landscapes import *
from models.Multi_dimensional_model import *

import tensorflow as tf
from tf_agents.drivers import dynamic_step_driver
from tf_agents.metrics import tf_metrics
from tf_agents.agents import tf_agent
from tf_agents.policies import random_tf_policy
from tf_agents.agents.ppo import ppo_policy, ppo_agent, ppo_utils
from tf_agents.environments import py_environment, tf_py_environment
from tf_agents.environments.utils import validate_py_environment
from tf_agents.drivers import dynamic_episode_driver
from tf_agents.networks import network, normal_projection_network
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.utils import common
from tf_agents.trajectories import time_step as ts
from tf_agents.specs import array_spec

In [23]:
RAA="UGCA" #alphabet
alphabet_len=len(RAA)
# TODO: UNDO THIS
# length=40
length=20
noise_alpha=1
generations = 10
experiment_batch_size = 1000
wt=generate_random_sequences(length,1,alphabet=RAA)[0]
landscape1=RNA_landscape(wt)
landscape2=RNA_landscape(wt)
noisy_landscape=Noise_wrapper(landscape1,
                              noise_alpha=noise_alpha,
                              always_costly=True)
initial_genotypes=list(set([wt]+[generate_random_mutant(wt,0.05,RAA) 
                                 for i in range(experiment_batch_size*10)]))[:experiment_batch_size]
noisy_landscape.reset()
noisy_landscape.measure_true_landscape(initial_genotypes)
noisy_landscape.natural_mode=False
noisy_landscape.local_mode=False
noisy_landscape.cost

def renormalize_moves(one_hot_input, rewards_output):
    """ensures that staying in place gives no reward"""
    zero_current_state = (one_hot_input - 1) * (-1)
    return np.multiply(rewards_output, zero_current_state)

def walk_away_renormalize_moves(one_hot_input, one_hot_wt, rewards_output):
    """ensures that moving toward wt is also not useful"""
    zero_current_state=(one_hot_input-1)*-1
    zero_wt=((one_hot_wt-1)*-1)
    zero_conservative_moves=np.multiply(zero_wt,zero_current_state)
    return np.multiply(rewards_output,zero_conservative_moves)

def get_all_singles_fitness(model,sequence,alphabet):
    prob_singles=np.zeros((len(alphabet),len(sequence)))
    for i in range(len(sequence)):
        for j in range(len(alphabet)):
            putative_seq=sequence[:i]+alphabet[j]+sequence[i+1:]
           # print (putative_seq)
            prob_singles[j][i]=model.get_fitness(putative_seq)
    return prob_singles

def get_all_mutants(sequence):
    mutants = []
    for i in range(sequence.shape[0]):
        for j in range(sequence.shape[1]):
            putative_seq = sequence.copy()
            putative_seq[:, j] = 0
            putative_seq[i, j] = 1
            mutants.append(putative_seq)
    return np.array(mutants)

def sample_greedy(matrix):
    i,j=matrix.shape
    max_arg=np.argmax(matrix)
    y=max_arg%j
    x=int(max_arg/j)
    output=np.zeros((i,j))
    output[x][y]=matrix[x][y]
    return output

def sample_multi_greedy(matrix):
    n = 5 # the number of base positions to greedily change
    max_args = np.argpartition(matrix.flatten(), -n)[-n:]
    i,j=matrix.shape
    output=np.zeros((i,j))
    for max_arg in max_args:
        y=max_arg%j
        x=int(max_arg/j)
        output[x][y]=matrix[x][y]
    return output

def sample_random(matrix):
    i,j=matrix.shape
    non_zero_moves=np.nonzero(matrix)
   # print (non_zero_moves)
    k=len(non_zero_moves)
    l=len(non_zero_moves[0])
    if k!=0 and l!=0:
        rand_arg=random.choice([[non_zero_moves[alph][pos] for alph in range(k)] for pos in range(l)])
    else:
        rand_arg=[random.randint(0,i-1),random.randint(0,j-1)]
    #print (rand_arg)
    y=rand_arg[1]
    x=rand_arg[0]
    output=np.zeros((i,j))
    output[x][y] = 1
    return output   

def action_to_scalar(matrix):
    matrix = matrix.ravel()
    for i in range(len(matrix)):
        if matrix[i] != 0:
            return i
    
def construct_mutant_from_sample(pwm_sample, one_hot_base):
    one_hot = np.zeros(one_hot_base.shape)
    one_hot += one_hot_base
    nonzero = np.nonzero(pwm_sample)
    nonzero = list(zip(nonzero[0], nonzero[1]))
    for nz in nonzero: # this can be problematic for non-positive fitnesses
        i, j = nz
        one_hot[:,j]=0
        one_hot[i,j]=1
    return one_hot

def best_predicted_new_gen(actor, genotypes, alphabet, pop_size):
    mutants = get_all_mutants(genotypes)
    one_hot_mutants = np.array([translate_string_to_one_hot(mutant, alphabet) for mutant in mutants])
    torch_one_hot_mutants = torch.from_numpy(np.expand_dims(one_hot_mutants, axis=0)).float()
    predictions = actor(torch_one_hot_mutants)
    predictions = predictions.detach().numpy()
    best_pred_ind = predictions.argsort()[-pop_size:]
    return mutants[best_pred_ind]

def make_one_hot_train_test(genotypes, model, alphabet):
    genotypes_one_hot = np.array([translate_string_to_one_hot(genotype, alphabet) for genotype in genotypes])
    genotype_fitnesses = []
    for genotype in genotypes:
        genotype_fitnesses.append(model.get_fitness(genotype))
    genotype_fitnesses = np.array(genotype_fitnesses)

    return genotypes_one_hot, genotype_fitnesses

In [24]:
# parameters for PPO Agent 
generations = 10
experiment_batch_size = 1000
global_step = tf.compat.v1.train.get_or_create_global_step()
optimizer = tf.keras.optimizers.Adam(1e-5)
optimizer.iterations = global_step

In [33]:
# environment for PPO Agent
class FitnessLandscapeEnvironment(py_environment.PyEnvironment):
    # Based on this: https://www.mikulskibartosz.name/how-to-create-an-environment-for-a-tensorflow-agent/
    def __init__(self, alphabet, seq_len, landscape, max_episodes):
        self.alphabet = alphabet
        self.alphabet_len = len(self.alphabet)
        self.landscape = copy.deepcopy(landscape)
        self.seq_len = seq_len
        self._action_spec = array_spec.BoundedArraySpec(
            shape=(1, 2), dtype=np.float32, minimum=0, 
            maximum=1, name='action_x')
        self._observation_spec = array_spec.BoundedArraySpec(
            shape=(self.alphabet_len, self.seq_len), dtype=np.float32, minimum=0, 
            maximum=1, name='observation')
        self._time_step_spec = ts.time_step_spec(self._observation_spec)
        self._state = translate_string_to_one_hot(wt, self.alphabet)
        self._episode_ended = False
        self.ctr = 0
        self.max_episodes = max_episodes
        self.seen_sequences = {}
        
    def _reset(self):
        self.ctr = 0
        # TODO: UNDO THIS
#         self._state = translate_string_to_one_hot(wt, self.alphabet)
        self._state = translate_string_to_one_hot(generate_random_sequences(length,1,alphabet=RAA)[0], self.alphabet)
        self._episode_ended = False
        return ts.restart(np.array(self._state, dtype=np.float32))
    
    def time_step_spec(self):
        return self._time_step_spec 

    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec
    
    def get_state_string(self):
        return translate_one_hot_to_string(self._state, self.alphabet)
    
    def _step(self, action):
        if self.ctr < self.max_episodes:
            self.ctr += 1
            action_one_hot = np.zeros((self.alphabet_len, self.seq_len))
#             print(action)
            if np.amax(action) > 1 or np.amin(action) < 0:
                return ts.termination(np.array(self._state, dtype=np.float32), 0)
            x, y = action[0]
            x, y = int(self.alphabet_len*x), int(self.seq_len*y)
            action_one_hot[x, y] = 1
            assert self._state.sum() == self._state.shape[1]
            if self._state[x, y] == 1:
                self._episode_ended = True
                return ts.termination(np.array(self._state, dtype=np.float32), 0)
            else:
                self._state = construct_mutant_from_sample(action_one_hot, self._state)
                state_string = translate_one_hot_to_string(self._state, self.alphabet)
                
                if state_string in self.seen_sequences:
                    return ts.termination(np.array(self._state, dtype=np.float32), 0)
                self.seen_sequences[state_string] = 1
                
                reward = self.landscape.get_fitness(state_string)
                assert self._state.sum() == self._state.shape[1]
                return ts.transition(np.array(self._state, dtype=np.float32), reward=reward)
        else:
            self._episode_ended = True
            assert self._state.sum() == self._state.shape[1]
            return ts.termination(np.array(self._state, dtype=np.float32), 0)

In [34]:
#max_iter = experiment_batch_size * generations 
max_iter = 10 ** 6
fle2 = FitnessLandscapeEnvironment(RAA, length, landscape1, max_iter)
print("starting validating environment...")
validate_py_environment(fle2, episodes=2)
print("done validating environment.")
fle = FitnessLandscapeEnvironment(RAA, length, landscape2, max_iter)
tf_env = tf_py_environment.TFPyEnvironment(fle)

starting validating environment...
done validating environment.


In [35]:
# specs 
time_step_spec = tf_env.time_step_spec()
observation_spec = tf_env.observation_spec()
action_spec = tf_env.action_spec()
alphabet_len = len(RAA)
seq_len = length

def BoostedEnvironment():
    return FitnessLandscapeEnvironment(RAA, length, landscape2, max_iter)

In [38]:
# coding=utf-8
# Copyright 2018 The TF-Agents Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

r"""Train and Eval PPO.
To run:
```bash
tensorboard --logdir $HOME/tmp/ppo/gym/HalfCheetah-v2/ --port 2223 &
python tf_agents/agents/ppo/examples/v2/train_eval.py \
  --root_dir=$HOME/tmp/ppo/gym/HalfCheetah-v2/ \
  --logtostderr
```
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import time

from absl import logging

import gin
import tensorflow as tf

from tf_agents.agents.ppo import ppo_agent
from tf_agents.drivers import dynamic_episode_driver
from tf_agents.environments import parallel_py_environment
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import actor_distribution_network
from tf_agents.networks import actor_distribution_rnn_network
from tf_agents.networks import value_network
from tf_agents.networks import value_rnn_network
from tf_agents.policies import policy_saver
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.utils import common


def train_eval(
    root_dir,
    env_name='HalfCheetah-v2',
    env_load_fn=BoostedEnvironment,
    random_seed=0,
    # TODO(b/127576522): rename to policy_fc_layers.
    actor_fc_layers=(200, 100),
    value_fc_layers=(200, 100),
    use_rnns=False,
    # Params for collect
    num_environment_steps=10000000,
    collect_episodes_per_iteration=30,
    num_parallel_environments=30,
    replay_buffer_capacity=1001,  # Per-environment
    # Params for train
    num_epochs=25,
    learning_rate=1e-5,
    # Params for eval
    num_eval_episodes=30,
    eval_interval=500,
    # Params for summaries and logging
    train_checkpoint_interval=500,
    policy_checkpoint_interval=500,
    log_interval=50,
    summary_interval=50,
    summaries_flush_secs=1,
    use_tf_functions=True,
    debug_summaries=True,
    summarize_grads_and_vars=False):
  """A simple train and eval for PPO."""
  if root_dir is None:
    raise AttributeError('train_eval requires a root_dir.')

  root_dir = os.path.expanduser(root_dir)
  train_dir = os.path.join(root_dir, 'train')
  eval_dir = os.path.join(root_dir, 'eval')
  saved_model_dir = os.path.join(root_dir, 'policy_saved_model')

  train_summary_writer = tf.compat.v2.summary.create_file_writer(
      train_dir, flush_millis=summaries_flush_secs * 1000)
  train_summary_writer.set_as_default()

  eval_summary_writer = tf.compat.v2.summary.create_file_writer(
      eval_dir, flush_millis=summaries_flush_secs * 1000)
  eval_metrics = [
      tf_metrics.AverageReturnMetric(buffer_size=num_eval_episodes),
      tf_metrics.AverageEpisodeLengthMetric(buffer_size=num_eval_episodes)
  ]

  global_step = tf.compat.v1.train.get_or_create_global_step()
  with tf.compat.v2.summary.record_if(
      lambda: tf.math.equal(global_step % summary_interval, 0)):
    tf.compat.v1.set_random_seed(random_seed)
    eval_tf_env = tf_py_environment.TFPyEnvironment(env_load_fn())
    tf_env = tf_py_environment.TFPyEnvironment(
        parallel_py_environment.ParallelPyEnvironment(
            [lambda: env_load_fn()] * num_parallel_environments))
    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

    if use_rnns:
      actor_net = actor_distribution_rnn_network.ActorDistributionRnnNetwork(
          tf_env.observation_spec(),
          tf_env.action_spec(),
          input_fc_layer_params=actor_fc_layers,
          output_fc_layer_params=None)
      value_net = value_rnn_network.ValueRnnNetwork(
          tf_env.observation_spec(),
          input_fc_layer_params=value_fc_layers,
          output_fc_layer_params=None)
    else:
      actor_net = actor_distribution_network.ActorDistributionNetwork(
          tf_env.observation_spec(),
          tf_env.action_spec(),
          fc_layer_params=actor_fc_layers)
      value_net = value_network.ValueNetwork(
          tf_env.observation_spec(), fc_layer_params=value_fc_layers)

    tf_agent = ppo_agent.PPOAgent(
        tf_env.time_step_spec(),
        tf_env.action_spec(),
        optimizer,
        actor_net=actor_net,
        value_net=value_net,
        num_epochs=num_epochs,
        debug_summaries=debug_summaries,
        summarize_grads_and_vars=summarize_grads_and_vars,
        train_step_counter=global_step)
    tf_agent.initialize()

    environment_steps_metric = tf_metrics.EnvironmentSteps()
    step_metrics = [
        tf_metrics.NumberOfEpisodes(),
        environment_steps_metric,
    ]

    train_metrics = step_metrics + [
        tf_metrics.AverageReturnMetric(
            batch_size=num_parallel_environments),
        tf_metrics.AverageEpisodeLengthMetric(
            batch_size=num_parallel_environments),
    ]

    eval_policy = tf_agent.policy
    collect_policy = tf_agent.collect_policy

    replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
        tf_agent.collect_data_spec,
        batch_size=num_parallel_environments,
        max_length=replay_buffer_capacity)

    train_checkpointer = common.Checkpointer(
        ckpt_dir=train_dir,
        agent=tf_agent,
        global_step=global_step,
        metrics=metric_utils.MetricsGroup(train_metrics, 'train_metrics'))
    policy_checkpointer = common.Checkpointer(
        ckpt_dir=os.path.join(train_dir, 'policy'),
        policy=eval_policy,
        global_step=global_step)
    saved_model = policy_saver.PolicySaver(
        eval_policy, train_step=global_step)

    train_checkpointer.initialize_or_restore()

    collect_driver = dynamic_episode_driver.DynamicEpisodeDriver(
        tf_env,
        collect_policy,
        observers=[replay_buffer.add_batch] + train_metrics,
        num_episodes=collect_episodes_per_iteration)

    def train_step():
      trajectories = replay_buffer.gather_all()
      return tf_agent.train(experience=trajectories)

    if use_tf_functions:
      # TODO(b/123828980): Enable once the cause for slowdown was identified.
      collect_driver.run = common.function(collect_driver.run, autograph=False)
      tf_agent.train = common.function(tf_agent.train, autograph=False)
      train_step = common.function(train_step)

    collect_time = 0
    train_time = 0
    timed_at_step = global_step.numpy()

    while environment_steps_metric.result() < num_environment_steps:
      global_step_val = global_step.numpy()
      if global_step_val % eval_interval == 0:
        metric_utils.eager_compute(
            eval_metrics,
            eval_tf_env,
            eval_policy,
            num_episodes=num_eval_episodes,
            train_step=global_step,
            summary_writer=eval_summary_writer,
            summary_prefix='Metrics',
        )

      start_time = time.time()
      collect_driver.run()
      collect_time += time.time() - start_time

      start_time = time.time()
      total_loss, _ = train_step()
      replay_buffer.clear()
      train_time += time.time() - start_time

      for train_metric in train_metrics:
        train_metric.tf_summaries(
            train_step=global_step, step_metrics=step_metrics)

      if global_step_val % log_interval == 0:
        logging.info('step = %d, loss = %f', global_step_val, total_loss)
        steps_per_sec = (
            (global_step_val - timed_at_step) / (collect_time + train_time))
        logging.info('%.3f steps/sec', steps_per_sec)
        logging.info('collect_time = {}, train_time = {}'.format(
            collect_time, train_time))
        with tf.compat.v2.summary.record_if(True):
          tf.compat.v2.summary.scalar(
              name='global_steps_per_sec', data=steps_per_sec, step=global_step)

        if global_step_val % train_checkpoint_interval == 0:
          train_checkpointer.save(global_step=global_step_val)

        if global_step_val % policy_checkpoint_interval == 0:
          policy_checkpointer.save(global_step=global_step_val)
          saved_model_path = os.path.join(
              saved_model_dir, 'policy_' + ('%d' % global_step_val).zfill(9))
          saved_model.save(saved_model_path)

        timed_at_step = global_step_val
        collect_time = 0
        train_time = 0

    # One final eval before exiting.
    metric_utils.eager_compute(
        eval_metrics,
        eval_tf_env,
        eval_policy,
        num_episodes=num_eval_episodes,
        train_step=global_step,
        summary_writer=eval_summary_writer,
        summary_prefix='Metrics',
    )
    print("EXITED")
    
    return tf_agent

logging.set_verbosity(logging.INFO)
tf.compat.v1.enable_v2_behavior()
agent = train_eval(
      '/Users/richard/Documents/Research/FLRL/dump',
      env_name='BoostedEnvironment',
      use_rnns=False,
      num_environment_steps=100000,
      collect_episodes_per_iteration=30,
      num_parallel_environments=4,
      replay_buffer_capacity=1000,
      num_epochs=25,
      num_eval_episodes=30)

INFO:absl:Spawning all processes.
INFO:absl:All processes started.
INFO:absl:Checkpoint available: /Users/richard/Documents/Research/FLRL/dump/train/ckpt-6000
INFO:absl:Checkpoint available: /Users/richard/Documents/Research/FLRL/dump/train/policy/ckpt-6000
INFO:absl:step = 6050, loss = 0.327176
INFO:absl:0.488 steps/sec
INFO:absl:collect_time = 0.913827657699585, train_time = 50.274866342544556
INFO:absl:step = 6100, loss = 0.593229
INFO:absl:258.382 steps/sec
INFO:absl:collect_time = 0.08973288536071777, train_time = 0.10377907752990723
INFO:absl:step = 6150, loss = 0.491860
INFO:absl:225.788 steps/sec
INFO:absl:collect_time = 0.1052849292755127, train_time = 0.11616206169128418
INFO:absl:step = 6200, loss = 0.431152
INFO:absl:198.255 steps/sec
INFO:absl:collect_time = 0.11707305908203125, train_time = 0.13512778282165527
INFO:absl:step = 6250, loss = 3.424420
INFO:absl:199.480 steps/sec
INFO:absl:collect_time = 0.14392495155334473, train_time = 0.10672688484191895
INFO:absl:step = 6

INFO:tensorflow:Assets written to: /Users/richard/Documents/Research/FLRL/dump/policy_saved_model/policy_000006500/assets


INFO:tensorflow:Assets written to: /Users/richard/Documents/Research/FLRL/dump/policy_saved_model/policy_000006500/assets
INFO:absl:step = 6550, loss = 0.457388
INFO:absl:301.313 steps/sec
INFO:absl:collect_time = 0.07382988929748535, train_time = 0.09211039543151855
INFO:absl:step = 6600, loss = 2.678300
INFO:absl:263.632 steps/sec
INFO:absl:collect_time = 0.08650803565979004, train_time = 0.1031501293182373
INFO:absl:step = 6650, loss = 2.355242
INFO:absl:248.940 steps/sec
INFO:absl:collect_time = 0.08457183837890625, train_time = 0.11627960205078125
INFO:absl:step = 6700, loss = 3.222640
INFO:absl:302.447 steps/sec
INFO:absl:collect_time = 0.07122921943664551, train_time = 0.09408926963806152
INFO:absl:step = 6750, loss = 0.195254
INFO:absl:277.279 steps/sec
INFO:absl:collect_time = 0.07951688766479492, train_time = 0.10080718994140625
INFO:absl:step = 6800, loss = 0.496647
INFO:absl:226.840 steps/sec
INFO:absl:collect_time = 0.10196495056152344, train_time = 0.1184544563293457
INFO

INFO:tensorflow:Assets written to: /Users/richard/Documents/Research/FLRL/dump/policy_saved_model/policy_000007000/assets


INFO:tensorflow:Assets written to: /Users/richard/Documents/Research/FLRL/dump/policy_saved_model/policy_000007000/assets
INFO:absl:step = 7050, loss = 1.937962
INFO:absl:266.433 steps/sec
INFO:absl:collect_time = 0.07274985313415527, train_time = 0.1149144172668457
INFO:absl:step = 7100, loss = 0.306742
INFO:absl:271.658 steps/sec
INFO:absl:collect_time = 0.07436990737915039, train_time = 0.10968518257141113
INFO:absl:step = 7150, loss = 0.339729
INFO:absl:288.976 steps/sec
INFO:absl:collect_time = 0.07502388954162598, train_time = 0.09800100326538086
INFO:absl:step = 7200, loss = 0.431425
INFO:absl:276.842 steps/sec
INFO:absl:collect_time = 0.08433699607849121, train_time = 0.09627127647399902
INFO:absl:step = 7250, loss = 0.269292
INFO:absl:287.913 steps/sec
INFO:absl:collect_time = 0.08380722999572754, train_time = 0.08985614776611328
INFO:absl:step = 7300, loss = 4.611185
INFO:absl:322.968 steps/sec
INFO:absl:collect_time = 0.06992697715759277, train_time = 0.08488702774047852
INF

...

INFO:tensorflow:Assets written to: /Users/richard/Documents/Research/FLRL/dump/policy_saved_model/policy_000048000/assets


INFO:tensorflow:Assets written to: /Users/richard/Documents/Research/FLRL/dump/policy_saved_model/policy_000048000/assets
INFO:absl:step = 48050, loss = 1.281160
INFO:absl:257.386 steps/sec
INFO:absl:collect_time = 0.09092068672180176, train_time = 0.10334014892578125
INFO:absl:step = 48100, loss = 0.592749
INFO:absl:256.048 steps/sec
INFO:absl:collect_time = 0.08860135078430176, train_time = 0.1066746711730957
INFO:absl:step = 48150, loss = 0.762767
INFO:absl:243.615 steps/sec
INFO:absl:collect_time = 0.09011578559875488, train_time = 0.11512613296508789
INFO:absl:step = 48200, loss = 6.125124
INFO:absl:242.147 steps/sec
INFO:absl:collect_time = 0.10020756721496582, train_time = 0.1062781810760498
INFO:absl:step = 48250, loss = 1.878250
INFO:absl:255.054 steps/sec
INFO:absl:collect_time = 0.09403872489929199, train_time = 0.10199785232543945
INFO:absl:step = 48300, loss = 1.391526
INFO:absl:265.453 steps/sec
INFO:absl:collect_time = 0.0825951099395752, train_time = 0.10576200485229492

INFO:tensorflow:Assets written to: /Users/richard/Documents/Research/FLRL/dump/policy_saved_model/policy_000048500/assets


INFO:tensorflow:Assets written to: /Users/richard/Documents/Research/FLRL/dump/policy_saved_model/policy_000048500/assets
INFO:absl:step = 48550, loss = 1.129066
INFO:absl:240.681 steps/sec
INFO:absl:collect_time = 0.10470914840698242, train_time = 0.10303497314453125
INFO:absl:step = 48600, loss = 14.142413
INFO:absl:241.037 steps/sec
INFO:absl:collect_time = 0.09703803062438965, train_time = 0.11039900779724121
INFO:absl:step = 48650, loss = 1.087065
INFO:absl:239.407 steps/sec
INFO:absl:collect_time = 0.09114694595336914, train_time = 0.11770200729370117
INFO:absl:step = 48700, loss = 3.698543
INFO:absl:221.039 steps/sec
INFO:absl:collect_time = 0.11099624633789062, train_time = 0.11520791053771973
INFO:absl:step = 48750, loss = 2.650952
INFO:absl:232.311 steps/sec
INFO:absl:collect_time = 0.10853195190429688, train_time = 0.10669708251953125
INFO:absl:step = 48800, loss = 0.968679
INFO:absl:273.325 steps/sec
INFO:absl:collect_time = 0.08434486389160156, train_time = 0.0985872745513

EXITED


In [39]:
be = BoostedEnvironment()
validate_py_environment(be, episodes=1)
tf_env = tf_py_environment.TFPyEnvironment(be)

# see how agent performs 
time_step = tf_env.reset()
for _ in range(50):
    tf_env.reset()
    episode_reward = 0
    episode_steps = 0
    while not tf_env.current_time_step().is_last():
        action = agent.policy.action(tf_env.current_time_step()).action
        print('predicted action', action)
        next_time_step = tf_env.step(action)
        episode_steps += 1
        print("Reward", next_time_step.reward.numpy())
        episode_reward += next_time_step.reward.numpy()
    print("Steps:", episode_steps, "Reward:", episode_reward)

predicted action tf.Tensor([[[0.32016593 0.52141124]]], shape=(1, 1, 2), dtype=float32)
Reward [0.00235294]
predicted action tf.Tensor([[[0.35313064 0.5240068 ]]], shape=(1, 1, 2), dtype=float32)
Reward [0.]
Steps: 2 Reward: [0.00235294]
predicted action tf.Tensor([[[0.47748858 0.4257298 ]]], shape=(1, 1, 2), dtype=float32)
Reward [0.02941176]
predicted action tf.Tensor([[[0.54622453 0.4379269 ]]], shape=(1, 1, 2), dtype=float32)
Reward [0.01882353]
predicted action tf.Tensor([[[0.47748858 0.4257298 ]]], shape=(1, 1, 2), dtype=float32)
Reward [0.]
Steps: 3 Reward: [0.0482353]
predicted action tf.Tensor([[[0.5850633 0.5429086]]], shape=(1, 1, 2), dtype=float32)
Reward [0.]
Steps: 1 Reward: [0.]
predicted action tf.Tensor([[[0.39333037 0.4595622 ]]], shape=(1, 1, 2), dtype=float32)
Reward [0.00235294]
predicted action tf.Tensor([[[0.44661033 0.4924605 ]]], shape=(1, 1, 2), dtype=float32)
Reward [0.]
Steps: 2 Reward: [0.00235294]
predicted action tf.Tensor([[[0.45497605 0.45990297]]], sha

predicted action tf.Tensor([[[0.45336407 0.47315836]]], shape=(1, 1, 2), dtype=float32)
Reward [0.02705882]
predicted action tf.Tensor([[[0.48258638 0.47715747]]], shape=(1, 1, 2), dtype=float32)
Reward [0.]
Steps: 2 Reward: [0.02705882]
predicted action tf.Tensor([[[0.608012   0.54048145]]], shape=(1, 1, 2), dtype=float32)
Reward [0.01294118]
predicted action tf.Tensor([[[0.57309616 0.52038664]]], shape=(1, 1, 2), dtype=float32)
Reward [0.]
Steps: 2 Reward: [0.01294118]
predicted action tf.Tensor([[[0.3862327  0.44728348]]], shape=(1, 1, 2), dtype=float32)
Reward [0.00823529]
predicted action tf.Tensor([[[0.43184984 0.4117758 ]]], shape=(1, 1, 2), dtype=float32)
Reward [0.]
Steps: 2 Reward: [0.00823529]
predicted action tf.Tensor([[[0.4680928  0.46774697]]], shape=(1, 1, 2), dtype=float32)
Reward [0.02235294]
predicted action tf.Tensor([[[0.4990568  0.49323973]]], shape=(1, 1, 2), dtype=float32)
Reward [0.]
Steps: 2 Reward: [0.02235294]
